In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
import os
import json
from datetime import datetime
from parse_process import parse_case_files, parse_metadata_files, parse_oneliner_files

In [3]:
cases_dir = os.listdir('./key_cases_scrapped_full/')
output_dir = './kc_processed/'

In [19]:
len(cases_dir)

149

In [27]:
def load_txt(file_path, mode):
    if mode == 'read':
        with open(file_path, 'r') as f:
            return f.read()
    elif mode == 'readlines':
        with open(file_path, 'r') as f:
            return f.readlines()
    else:
        raise ValueError('Invalid mode')

def save_json(file_path, data):
    with open(file_path, 'w') as f:
        json.dump(data, f)

In [31]:
cutoff_date_str = '31/10/2023'
cutoff_date = datetime.strptime(cutoff_date_str, "%d/%m/%Y").date()

before_cutoff = 0
after_cutoff = 0

for case_dir in cases_dir:
    #print('processing:', case_dir)
    case_i = './key_cases_scrapped_full/' + case_dir
    files = sorted(os.listdir(case_i))
    #print(files)
    year = files[0].split('-')[0]
    #print(year)
    case_data = {}
    
    # parse case text
    case_i_text = load_txt(case_i + f'/{files[2]}', 'read')
    case_i_text_parsed = parse_case_files(case_i_text)
    case_data['case_text'] = case_i_text_parsed

    if 'THE FACTS' not in case_i_text_parsed or len(case_i_text_parsed['THE FACTS']) == 0:
        print('skipping:', case_dir, 'no facts')
        continue

    # parse metadata
    case_i_metadata = load_txt(case_i + f'/{files[0]}', 'readlines')
    extracted_metadata = parse_metadata_files(case_i_metadata)
    case_data['metadata'] = extracted_metadata
    if 'Judgment Date' not in extracted_metadata and 'Decision Date' not in extracted_metadata:
        print('skipping:', case_dir, 'no date')
        continue
    else:
        cutoff_date_str = '01/10/2023'
        date_key = "Judgment Date" if "Judgment Date" in extracted_metadata else "Decision Date" if "Decision Date" in extracted_metadata else None
        file_date = datetime.strptime(extracted_metadata[date_key], "%d/%m/%Y").date()
        
        # Compare the file date with the cutoff date
        if file_date > cutoff_date:
            case_data['cut-off'] = 'after'
            after_cutoff += 1
        else:
            case_data['cut-off'] = 'before'
            before_cutoff += 1


    # parse one-liner
    case_i_oneliner = load_txt(case_i + f'/{files[1]}', 'readlines')
    extracted_oneliner = parse_oneliner_files(case_i_oneliner)
    case_data['one-liner'] = extracted_oneliner

    #print('---------------------')
    save_json(output_dir + case_dir + '.json', case_data)

skipping: 001-203699 no facts
skipping: 001-189019 no facts
skipping: 001-222780 no facts
skipping: 001-224629 no facts
skipping: 001-186434 no facts
skipping: 001-231227 no facts
skipping: 001-194618 no date
skipping: 001-222410 no facts
skipping: 001-184438 no facts
skipping: 001-195544 no date
skipping: 001-234982 no facts
skipping: 001-223932 no facts
skipping: 001-222750 no date
skipping: 001-182452 no date
skipping: 001-219559 no facts
skipping: 001-219707 no facts
skipping: 001-180276 no facts
skipping: 001-222660 no facts
skipping: 001-203370 no facts
skipping: 001-235426 no facts
skipping: 001-223924 no facts
skipping: 001-181585 no facts
skipping: 001-233937 no facts
skipping: 001-194523 no date


In [32]:
processed_cases = os.listdir(output_dir)
len(processed_cases)

125

In [33]:
before_cutoff, after_cutoff

(107, 18)